# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.8 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
import matplotlib.pyplot as plt
# Import models you're considering
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-268699876.py:17: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


## Model Selection


We consider a neural network or an LSTM (those were used in the models in the literature review)



## Feature Engineering

Three feature engeneering steps were performed: Instead of using the max und min temperature, the span of temperature of that day was used. --> Tells if the weather was stable or fluctuating that day

The interaction between the average Temperature and the humidity is used -->There is a correlation between these two parameters


The pressure change is also interesting, a decreasing pressure indicates a low pressure area, an increasing pressure indicates a high pressure area


In [24]:
# Load the dataset
# Replace 'your_dataset.csv' with the path to your actual dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dobersdorf_all_cut.csv', sep=";", decimal=",", header=0)

# Perform any feature engineering steps
# Example: df['new_feature'] = df['feature1'] + df['feature2']
df['temp_range'] = df['Tmax'] - df['Tmin']
df['temp_hum_interaction'] = df['Tavg'] * df['Hum']
##.diff() calculates the difference to the value the day before
df['pressure_change'] = df['Pres'].diff()

# Feature and target variable selection
X = df[['Tavg', 'temp_range','temp_hum_interaction','Sun','Pres','pressure_change','Vis', 'Tmin']]
y = df['Produktion(kWH)']
###values between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y = np.array(y).reshape(-1, 1)
y_scaled = scaler.fit_transform(y)


split_index = int(len(df) * 0.8)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]

# Split the dataset
##Splitting time series data random is problematic
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
print(X_train)
print(y_train)

###here we had to exclude missing values
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)

print("X nan:", np.isnan(X_train).any())
print("y nan:", np.isnan(y_train).any())
print("X inf:", np.isinf(X_train).any())
print("y inf:", np.isinf(y_train).any())

###here we had to exclude missing values
X_test= np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)

print("X nan:", np.isnan(X_train).any())
print("y nan:", np.isnan(y_train).any())
print("X inf:", np.isinf(X_train).any())
print("y inf:", np.isinf(y_train).any())

673
169
673
169
[[0.82040872 0.33918129 0.79795612 ...        nan 0.60631493 0.88501742]
 [0.7607254  0.38011696 0.76776064 ... 0.65543365 0.55495684 0.76655052]
 [0.7711463  0.54385965 0.74285803 ... 0.60309996 0.66921874 0.7456446 ]
 ...
 [0.63594532 0.60818713 0.57047326 ... 0.52995472 0.65903628 0.57142857]
 [0.66274191 0.67836257 0.57353263 ... 0.48719959 0.87209257 0.58536585]
 [0.67668155 0.50292398 0.58493365 ... 0.56739812 0.9432664  0.65853659]]
[[0.6277709 ]
 [0.61471813]
 [0.73905705]
 [0.31799257]
 [0.66310341]
 [0.59120063]
 [0.50602003]
 [0.18780241]
 [0.44829526]
 [0.84741758]
 [0.57297176]
 [0.48992911]
 [0.58017329]
 [0.55800608]
 [0.55946889]
 [0.28198492]
 [0.45617194]
 [0.36975357]
 [0.41172499]
 [0.55924384]
 [0.63733543]
 [0.44210645]
 [0.3148419 ]
 [0.58512434]
 [0.64037358]
 [0.64217396]
 [0.46888714]
 [0.19162822]
 [0.60852931]
 [0.61764375]
 [0.56194441]
 [0.72589175]
 [0.12996512]
 [0.33667154]
 [0.43029144]
 [0.63474738]
 [0.49848093]
 [0.73624395]
 [0.5748

## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]


In [ ]:
# Implement hyperparameter tuning
# Example using GridSearchCV with a DecisionTreeClassifier
# param_grid = {'max_depth': [2, 4, 6, 8]}
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)



###Test
# Test- und Trainingsdaten als fortlaufende Index-Serien
TIME_STEPS = 5
#
#y_train = y_train.reset_index(drop=True)
#y_test  = y_test.reset_index(drop=True)
X_test_np = np.array(X_test)
y_test_np = y_test.reset_index(drop=True).to_numpy()

def create_lstm_dataset(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    X = np.array(X)
    y = np.array(y)  # jetzt 0…n-1
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

X_train_3d, y_train_3d = create_lstm_dataset(X_train, y_train, TIME_STEPS)
print("X_train_3d shape:", X_train_3d.shape)
print("y_train_3d shape:", y_train_3d.shape)

print(np.isnan(X_train_3d).any())
print(np.isinf(X_train_3d).any())

print(np.isnan(y_train_3d).any())
print(np.isinf(y_train_3d).any())

def build_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('units', 8, 32, step=16),
        input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])
    ))
    model.add(Dense(1))
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', [0.000001, 0.00001])),
        loss='mse',
        metrics=['mae']
    )
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='lstm_tuning',
    project_name='time_series_lstm'
)

# === Suche starten ===
tuner.search(X_train_3d, y_train_3d, epochs=10, validation_split=0.2, batch_size=8)

# Beste Hyperparameter ausgeben
best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"Beste Units: {best_hps.get('units')}, Beste Learning Rate: {best_hps.get('learning_rate')}")


model = build_model(best_hps)
history = model.fit(X, y, epochs=50, validation_split=0.2)




X_train_3d shape: (668, 5, 8)
y_train_3d shape: (668,)
False
False
False
False
Reloading Tuner from lstm_tuning/time_series_lstm/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
128               |128               |units
0.0001            |0.0001            |learning_rate

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1308.5209 - mae: 27.5079 - val_loss: 1854.4811 - val_mae: 33.9072
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1480.3474 - mae: 29.1347 - val_loss: 1678.1913 - val_mae: 31.5971
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1283.9006 - mae: 26.3519 - val_loss: 1262.3590 - val_mae: 26.3440
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 808.9385 - mae: 21.1181 - val_loss: 1076.6283 - val_mae: 24.6226
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 776.6734 - mae: 21.4112 - val_loss: 1010.7936 - val_mae: 24.1506
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 693

RuntimeError: Number of consecutive failures exceeded the limit of 3.


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [25]:


def create_lstm_dataset(X, y, time_step=1):
      Xs, ys = [], []
      X = np.array(X)
      y = np.array(y)  # jetzt 0…n-1
      for i in range(len(X) - time_step):
        Xs.append(X[i:i+time_step,:])
        ys.append(y[i+time_step])
      return np.array(Xs), np.array(ys)

time_step = 10
X,y=create_lstm_dataset(X_train,y_train, time_step)
print(len(X))
print(len(y))


# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 8)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=20, batch_size=32)



663
663
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0755
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0346
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0329
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0348
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0349
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0316
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0324
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0324
Epoch 9/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0321
Epoch 10/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0310
Epoch 11/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0304
Epoch 12/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0300
Epoch 13/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0315
Epoch 14/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0279
Epoch 15/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0316
Epoch 16/20
21/

## Evaluation Metrics

[Clearly specify which metrics you'll use to evaluate the model performance, and why you've chosen these metrics.]


In [ ]:
# Evaluate the model using your chosen metrics
# Example for classification
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))

# Example for regression
# mse = mean_squared_error(y_test, y_pred)

# Your evaluation code here


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]


In [ ]:
# Comparative Analysis code (if applicable)
# Example: comparing accuracy of the baseline model and the new model
# print(f"Baseline Model Accuracy: {baseline_accuracy}, New Model Accuracy: {new_model_accuracy}")
